In [27]:
import pandas as pd
import numpy as np

from datetime import datetime 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, 
                             LearningRateScheduler, TensorBoard)
from keras.optimizers import SGD, Adam

38898

In [57]:
test

,Unnamed: 0,Access,DMA,Date Aired,Estimate,GRP,Impression,Impression (000),Impressions,Length,Market,Media,Spot Cost,Spot Type,Station ID,Time Aired,DOW,daypart
0,0,30GD,900,2018-12-31,Q119,0.000,NaN,0.0,NaN,30,National Network,TV,56.0,INV,37,00:06:00,Monday,Late Night
1,1,30TN,900,2018-12-31,Q119,0.000,NaN,0.0,NaN,30,Cable,TV,56.0,INV,202,00:07:00,Monday,Late Night
2,2,30PM,900,2018-12-31,Q119,0.016,NaN,27.2,NaN,30,Cable,TV,142.4,INV,253,00:09:00,Monday,Late Night
3,3,30PM,900,2018-12-31,Q119,0.128,NaN,158.4,NaN,30,Cable,TV,64.0,INV,340,00:09:00,Monday,Late Night
4,4,30GD,900,2018-12-31,Q119,0.000,NaN,0.0,NaN,30,Cable,TV,56.0,INV,168,00:10:00,Monday,Late Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683324,50571,30GD,900,2019-07-28,Q319,0.000,NaN,NaN,0.0,30,Cable,TV,80.0,INV,50,23:53:00,Sunday,Late Fringe
683325,50572,30GD,900,2019-07-28,Q319,0.000,NaN,NaN,0.0,30,Cable,TV,48.0,INV,196,23:54:00,Sunday,Late Fringe
683326,50573,30PT,900,2019-07-28,Q319,0.000,NaN,NaN,0.0,30,Cable,TV,48.0,INV,313,23:55:00,Sunday,Late Fringe
683327,50574,30PM,900,2019-07-28,Q319,0.000,NaN,NaN,0.0,30,Cable,TV,80.0,INV,39,23:56:00,Sunday,Late Fringe


In [9]:
dat = pd.read_csv('../Data/dat_features.csv')

dat = dat.loc[:, "Access":]

In [10]:
unseen = dat[(dat.Impressions.eq(0.0)) & (dat.GRP.eq(0.0)) | (dat.active_flag.eq(1))]
training = dat[dat.Impressions > 0.0]

In [11]:
dat.columns

Index(['Access', 'DMA', 'Date_Aired', 'Estimate', 'GRP', 'Impressions',
       'Length', 'Market', 'Media', 'Spot_Cost', 'Spot_Type', 'Station_ID',
       'Time_Aired', 'DOW', 'daypart', 'Hours', 'Month', 'Quarter', 'creative',
       'is_outlier', 'Station_ID_', 'bins', 'bin_1', 'bin_2', 'bin_3', 'bin_4',
       'bin_5', 'BP', 'DC', 'DE', 'DP', 'GD', 'GX', 'PL', 'PM', 'PN', 'PT',
       'SR', 'SV', 'TN', 'VE', 'Q119', 'Q219', 'Q319', 'Q419', 'Cable',
       'DirecTV', 'Dish_Network', 'National_Network', 'Over-the-top_content',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday', 'Q1', 'Q2', 'Q3', 'Q4', 'Daytime', 'Early_Fringe',
       'Late_Fringe', 'Late_Night', 'Morning', 'Overnight', 'Primetime',
       'midnight', 'one_am', 'two_am', 'three_am', 'four_am', 'five_am',
       'six_am', 'seven_am', 'eight_am', 'nine_am', 'ten_am', 'eleven_am',
       'noon', 'one_pm', 'two_pm', 'three_pm', 'four_pm', 'five_pm', 'six_pm',
       'seven_pm', 'eig

In [44]:
labels = [
        'Q119', 'Q219',
        'Q319', 'Q419',
        'BP', 'DC', 'DE', 'DP',
        'GD', 'GX', 'PL', 'PM', 
        'PN', 'PT', 'SR', 'SV', 
        'TN', 'VE',
        "Length",
        "Spot_Cost",
        "Cable", 
        "DirecTV",
        "Dish_Network",
        "National_Network",
        "Over-the-top_content",
        'Q1', 'Q2', 'Q3', 'Q4',
        'bin_1', 'bin_2',                
        'bin_3', 'bin_4', 'bin_5',
        'Daytime', 'Early_Fringe',
        'Late_Fringe', 'Late_Night', 
        'Morning', 'Overnight', 
        'Primetime'
    ]

X = training.loc[:, labels]
y = training.loc[:, "Impressions"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [55]:
dat['Impressions']

0           0.0
1           0.0
2          27.2
3         158.4
4           0.0
          ...  
683324      NaN
683325      NaN
683326      NaN
683327      NaN
683328      NaN
Name: Impressions, Length: 683329, dtype: float64

In [45]:
dim = len(X_train.columns)

In [46]:
filepath = '../nn_files/weights.hdf5'
logspath = '../nn_files/logs/scalars/' + datetime.now().strftime("%Y%m%d-%H%M%S")

def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return float(0.001 * tf.math.exp(0.1*(10-epoch)))
    
callbacks = [TensorBoard(log_dir=logspath),
             EarlyStopping(monitor='val_mean_squared_error', patience=15, mode='min'),
             ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, mode='min', period=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_mean_squared_error', patience=2, verbose=1, factor=0.20, min_lr=0.000000000001, mode='min')]


model = Sequential()
model.add(Dense(1024, input_dim=dim, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1))

ad = Adam(lr = 0.00001, 
         amsgrad = True)
model.compile(loss='mean_squared_error', 
              optimizer=ad,
              metrics=['mean_squared_error'])

model.fit(X_train, y_train, 
          epochs = 100, validation_split=0.2, callbacks=callbacks)

Train on 2744 samples, validate on 686 samples
Epoch 1/100
 768/2744 [=======>......................] - ETA: 2s - loss: 3813.3899 - mean_squared_error: 3813.3896 

c:\users\alex\appdata\local\programs\python\python38\lib\site-packages\keras\callbacks\callbacks.py:92: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.363875). Check your callbacks.
  warnings.warn(


2744/2744 [==============================] - 1s 538us/step - loss: 3857.9167 - mean_squared_error: 3857.9177 - val_loss: 3683.2982 - val_mean_squared_error: 3683.2981

Epoch 00001: val_mean_squared_error improved from inf to 3683.29810, saving model to ../nn_files/weights.hdf5
Epoch 2/100
2744/2744 [==============================] - 1s 233us/step - loss: 3465.2601 - mean_squared_error: 3465.2598 - val_loss: 3222.9565 - val_mean_squared_error: 3222.9563

Epoch 00002: val_mean_squared_error improved from 3683.29810 to 3222.95630, saving model to ../nn_files/weights.hdf5
Epoch 3/100
2744/2744 [==============================] - 1s 236us/step - loss: 2993.9725 - mean_squared_error: 2993.9724 - val_loss: 2847.1556 - val_mean_squared_error: 2847.1558

Epoch 00003: val_mean_squared_error improved from 3222.95630 to 2847.15576, saving model to ../nn_files/weights.hdf5
Epoch 4/100
2744/2744 [==============================] - 1s 232us/step - loss: 2791.6058 - mean_squared_error: 2791.6060 - val_l

KeyboardInterrupt: 

In [34]:
preds = model.predict(X_test)

In [35]:
print(np.sqrt(MSE(y_test, preds)))

47.09875159212858
